In [2]:
%reload_ext autoreload
%autoreload 2

# Setup colored logging

In [3]:
import logging

import colorlog

handler = colorlog.StreamHandler()
fmt = "%(log_color)s%(levelname)s:%(name)s:%(message)s"
formatter = colorlog.ColoredFormatter(
    fmt,
    log_colors={
        "DEBUG": "purple",
        "INFO": "green",
        "WARNING": "yellow",
        "ERROR": "red",
        "CRITICAL": "red,bg_white",
    },
)
handler.setFormatter(formatter)
logging.basicConfig(level=logging.DEBUG, handlers=[handler])
logger = logging.getLogger("mtg-ai")
logger.propagate = True

# Dataset Builder

In [ ]:
from mtg_ai.cards import MTGDatabase
row = MTGDatabase()
row.df.loc[row.df["name"] == "Ajani, Nacatl Pariah // Ajani, Nacatl Avenger"]

In [ ]:
from mtg_ai.cards.training_data_builder import build_question_answer_datasets

row = MTGDatabase()
build_question_answer_datasets(row, recreate=True)

# Training

In [ ]:
from mtg_ai.ai import ModelAndTokenizer, MTGCardAITrainer

model = ModelAndTokenizer.UNSLOTH_LLAMA_3_2_3B_INSTRUCT_Q8

# dataset = "data/question_answer_dataset.csv"
# loader = MTGCardAITrainingDatasetLoader(dataset, model.tokenizer)
gguf_file = None  # model.gguf_file
ai_trainer = MTGCardAITrainer(model_name=model.value, gguf_file=gguf_file)

ai_trainer.train(resume_from_checkpoint=True)

# Combine Lora with model

In [ ]:
from pathlib import Path

from mtg_ai.ai import ModelAndTokenizer, save_combined_model

model_name = ModelAndTokenizer.BARTOWSKI_LLAMA_3_2_8B_INSTRUCT_Q4_K_L
model_dir = Path("./results").resolve()

save_combined_model(
    model_name=model_name.value,
    tokenizer_name=model_name.tokenizer,
    gguf_file=model_name.gguf_file,
    model_dir=model_dir,
)

# Inference

In [1]:
import logging
import colorlog
from pathlib import Path

# handler = colorlog.StreamHandler()
# fmt = "%(log_color)s%(levelname)s:%(name)s:%(message)s"
# formatter = colorlog.ColoredFormatter(
#     fmt,
#     log_colors={
#         "DEBUG": "purple",
#         "INFO": "green",
#         "WARNING": "yellow",
#         "ERROR": "red",
#         "CRITICAL": "red,bg_white",
#     },
# )
# handler.setFormatter(formatter)
# logging.basicConfig(level=logging.DEBUG, handlers=[handler])

from mtg_ai.ai import MTGAIRunner

ai_model_name: str = "./results"
rag_embedding_model_name: str = "sentence-transformers/all-MiniLM-L6-v2"
runner = MTGAIRunner(
    ai_model_name=ai_model_name, 
    rag_embedding_model_name=rag_embedding_model_name
)

test_questions = [
    "What is the converted mana cost of Acquisitions Expert?",
    "What is the converted mana cost of Loch Korrigan?",
    "What is the type of Loch Korrigan?",
    "What is the text of Loch Korrigan?",
    "What is the text of Tarmogoyf?",
    "What is the type of Ajani, Nacatl Pariah",
    "What is the text of Ajani, Nacatl Pariah",
    "What is the cmc of Ajani, Nacatl Pariah",
    "What is the text of Ajani, Nacatl Avenger",
    "What is the type of Ajani, Nacatl Avenger",
]
max_new_tokens = 500
top_k = 3

for tq in test_questions:
    print(tq)
    runner.run(tq, max_new_tokens=500, filters=None, top_k=3)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.5: Fast Llama patching. Transformers = 4.46.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 3080. Max memory: 12.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.10.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Building Documents: 0it [00:00, ?it/s]

Batches:   0%|          | 0/635 [00:00<?, ?it/s]

What is the converted mana cost of Acquisitions Expert?


...

2
What is the text of Ajani, Nacatl Avenger


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[+2]: Put a +1/+1 counter on each Cat you control.\n[0]: Create a 2/1 white Cat Warrior creature token. When you do, if you control a red permanent other than Ajani, Nacatl Avenger, he deals damage equal to the number of creatures you control to any target.\n[−4]: Each opponent chooses an artifact, a creature, an enchantment, and a planeswalker from among the nonland permanents they control, then sacrifices the rest.
What is the type of Ajani, Nacatl Avenger


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Legendary Creature — Cat Warrior


# Experimenting 